In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# -------------------------------
# Adam オプティマイザ
# -------------------------------
class Adam:
    def __init__(self, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps

        self.m = None
        self.v = None
        self.t = 0

    def update(self, params, grads):
        if self.m is None:
            # パラメータと同じ形状でモーメント(m)を初期化
            self.m = np.zeros_like(params)
        if self.v is None:
            # パラメータと同じ形状で二乗勾配のモーメント(v)を初期化
            self.v = np.zeros_like(params)

        self.t += 1

        # モーメントの更新
        # m: 勾配の指数移動平均（慣性項）
        self.m = self.beta1 * self.m + (1 - self.beta1) * grads
        # v: 勾配の二乗の指数移動平均（適応的学習率項）
        self.v = self.beta2 * self.v + (1 - self.beta2) * (grads ** 2)

        # バイアス補正 (初期のバイアスを解消)
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)

        # パラメータ更新
        # Adamの更新式: params -= lr * m_hat / (sqrt(v_hat) + eps)
        params -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)
        return params

# -------------------------------
# 最適化したい関数（例：二次関数）
# -------------------------------
def f(x, y):
    """目的関数: f(x, y) = x^2 + 3y^2 + 2x + 4y"""
    # 最小値は f'(x)=0, f'(y)=0 より x = -1, y = -4/6 = -2/3
    return x**2 + 3*y**2 + 2*x + 4*y

# 勾配
def grad_f(x, y):
    """偏微分 (勾配) ベクトル"""
    df_dx = 2*x + 2
    df_dy = 6*y + 4
    return np.array([df_dx, df_dy])


# -------------------------------
# 最適化ループ
# -------------------------------
adam = Adam(lr=0.01)
params = np.array([4.0, 4.0])  # 初期値 (x=4, y=4)

trajectory = []  # 軌跡保存

print(f"初期値: x={params[0]:.2f}, y={params[1]:.2f}, f(x,y)={f(params[0], params[1]):.2f}")

for step in range(1000):
    grads = grad_f(params[0], params[1])
    params = adam.update(params, grads)
    trajectory.append(params.copy())

    if (step + 1) % 100 == 0:
        loss = f(params[0], params[1])
        print(f"Step {step+1}: x={params[0]:.4f}, y={params[1]:.4f}, Loss={loss:.4f}")

trajectory = np.array(trajectory)

print(f"\n最終到達点: x={params[0]:.4f}, y={params[1]:.4f}")
print(f"理論上の最小値: x=-1.0, y=-0.6667")

# -------------------------------
# ３Ｄ可視化
# -------------------------------
fig = plt.figure(figsize=(12, 5))

# ======= 3D Surface Plot ======
ax = fig.add_subplot(1, 2, 1, projection='3d')

# 曲面の定義
X = np.linspace(-5, 5, 200)
Y = np.linspace(-5, 5, 200)
X, Y = np.meshgrid(X, Y)
Z = f(X, Y)

# 曲面の描画 (透明度を設定)
ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.6)

# 軌跡の描画 (Z軸の高さは、軌跡上のf(x,y)の値)
Z_traj = f(trajectory[:, 0], trajectory[:, 1])
ax.plot(trajectory[:, 0], trajectory[:, 1], Z_traj, color='red', linewidth=3, label='Adam Path')
ax.scatter(trajectory[0, 0], trajectory[0, 1], Z_traj[0], color='black', s=50, label='Start')
ax.scatter(trajectory[-1, 0], trajectory[-1, 1], Z_traj[-1], color='cyan', s=50, label='End')

ax.set_title("Adam Optimization Path (3D Surface)", fontsize=14)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("f(x, y)")
ax.legend()


# ======= Contour Plot ======
ax2 = fig.add_subplot(1, 2, 2)

# 等高線の描画
contour = ax2.contour(X, Y, Z, levels=30, cmap='viridis')
ax2.clabel(contour, inline=1, fontsize=8) # ラベル表示

# 軌跡の描画
ax2.plot(trajectory[:, 0], trajectory[:, 1], color='red', linewidth=2, label='Adam Path')
ax2.scatter(trajectory[0, 0], trajectory[0, 1], color='black', s=50, label='Start')
ax2.scatter(trajectory[-1, 0], trajectory[-1, 1], color='cyan', s=50, label='End')

ax2.set_title("Optimization Trajectory (Contour)", fontsize=14)
ax2.set_xlabel("x")
ax2.set_ylabel("y")
ax2.legend()
ax2.set_aspect('equal', adjustable='box') # 縦横比を等しくして歪みを防ぐ

plt.suptitle("Adam Optimizer Visualization on $f(x, y) = x^2 + 3y^2 + 2x + 4y$", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # タイトル分のスペースを確保
plt.show()

SyntaxError: invalid non-printable character U+00A0 (ipython-input-3575332075.py, line 64)